### Development Method

1. get public transport location data - NAPTAN a) Clean it if necessary
2. get population location data -  LSOA from ONS
3. use Fiona to read location data
4. limit to one or two locations, e.g. London and a more rural area
5. draw Euclidean Buffers around LSOA polygon centre points
6. find number of public transport stops in the polygon with “points in polygons” approach

In [11]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [3]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import requests
import json
import pyproj

In [4]:
def geo_df_from_csv(path_to_csv, geom_x, geom_y, delim='\t', crs ="EPSG:4326"):
    """Function to create a Geo-dataframe from a csv file.
        The process goes via Pandas
    
        Arguments:
            path_to_csv (string): path to the txt/csv containing geo data
                to be read
            delimiter (string): the seperator in the csv file e.g. "," or "\t" 
            geom_x (string):name of the column that contains the longitude data
            geom_y (string):name of the column that contains the latitude data
            
        Returns:
            Geopandas Dataframe
            """
    pd_df = pd.read_csv(path_to_csv, delim)
    geometry = [Point(xy) for xy in zip(pd_df[geom_x], pd_df[geom_y])]
    geo_df = gpd.GeoDataFrame(pd_df, geometry=geometry)
    geo_df.crs = crs
    return geo_df


stops_path = (os.path.join
              (os.getcwd(),
               'data',
               'Stops.txt'))

stops_geo_df = (geo_df_from_csv(path_to_csv=stops_path,
                            delim='\t',
                            geom_x='stop_lon',
                            geom_y='stop_lat'))
stops_geo_df.sample(15)

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,vehicle_type,geometry
199391,340001869OUT,oxfgdwmg,"Wootton, The Duke of Marlborough PH (o/s)",51.86896,-1.37794,NaN,3.0,POINT (-1.37794 51.86896)
79923,150044006002,esxajpmt,"Jaywick, Tyndale Drive (opp)",51.78601,1.11825,NaN,3.0,POINT (1.11825 51.78601)
357341,6290LV01,95624982,"Kirkton, Village Primary School (after)",55.88673,-3.54526,NaN,3.0,POINT (-3.54526 55.88673)
333772,5810AWO26316,swagamj,"Cwmdu, Weig Gardens",51.64107,-3.96506,NaN,3.0,POINT (-3.96506 51.64107)
191210,3300BA0083,ntsdwtaw,"Misterton, Stockwith Road (S-bound)",53.43546,-0.82856,NaN,3.0,POINT (-0.82856 53.43546)
375910,6700713113,45328532,"Skerray, Free Church (nr)",58.52955,-4.28117,NaN,3.0,POINT (-4.28117 58.52955)
190197,3290YYA01072,32901072,"Poppleton Park, Kyle Way (opp)",53.97830,-1.13682,NaN,3.0,POINT (-1.13682 53.97830)
288452,460000239,wildwjpa,"Chitterne, The Kings Arms (E-bound)",51.19372,-2.01561,NaN,3.0,POINT (-2.01561 51.19372)
169663,2800TX00144,NaN,"Birkenhead, Exmouth Street (Taxi Rank)",53.38955,-3.03049,NaN,21.0,POINT (-3.03049 53.38955)
86401,1600GLC048,glogjgmg,"Southrop, Southrop School (E-bound)",51.72977,-1.70966,NaN,3.0,POINT (-1.70966 51.72977)


In [5]:
def geo_df_from_geospatialfile(path_to_file, crs="EPSG:4326"):
    
    """Function to create a Geo-dataframe from a csv file.
        The process goes via Pandas
    
        Arguments:
            path_to_file (string): path to the geojson, shp and other geospatial data files

        Returns:
            Geopandas Dataframe
            """
    geo_df = gpd.read_file(path_to_file)
    if geo_df.crs != crs:
        geo_df = geo_df.to_crs("EPSG:4326")
    return geo_df
        

In [6]:
def find_points_in_poly(geo_df, polygon_obj):
    """Find points in polygon using geopandas' spatial join.
        Then drops all rows where the point is not in the polygon
        (based on column index_right not being NaN). Finally it
        drop all column names from that were created in the join,
        leaving only the columns of the original geo_df
        
        Arguments:
            geo_df (string): name of a geo pandas dataframe
            polygon_obj (string): a geopandas dataframe with a polygon column
            
        Returns:
            A geodata frame with the points inside the supplied polygon"""
    wanted_cols = geo_df.columns.to_list()
    joined_df = (gpd.sjoin
                 (geo_df,
                  polygon_obj,
                  how='left',
                  op='within'))
    filtered_df = (joined_df
                   [joined_df
                    ['index_right'].notna()])
    filtered_df = filtered_df[wanted_cols]
    return filtered_df




In [11]:
greater_london_path = ((os.path.join
                                (os.getcwd(),
                                 'data',
                                 'greater_london.geojson')))

greater_london_geo_df = geo_df_from_geospatialfile(greater_london_path)

greater_london_geo_df.head()

# Creating a Geo Dataframe of only stops in London
london_stops_geo_df = (find_points_in_poly
                       (geo_df=stops_geo_df,
                        polygon_obj=greater_london_geo_df))

london_stops_geo_df.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,vehicle_type,geometry
76301,150012891S,esxjdtjp,"Grange Hill, Stradbroke Park (adj)",51.60482,0.07290,NaN,3.0,POINT (0.07290 51.60482)
79876,150042023001,esxatmga,"Grange Hill, Tudor Crescent (adj)",51.60665,0.08303,NaN,3.0,POINT (0.08303 51.60665)
122161,210021803340,hrtajatj,"Batchworth Heath, Mount Vernon Hospital (nr)",51.61460,-0.45066,NaN,3.0,POINT (-0.45066 51.61460)
123431,210021001322,hrtgtdad,"Dancers Hill, The Shires (nr)",51.66453,-0.20933,NaN,3.0,POINT (-0.20933 51.66453)
134927,2400107805,kntjwmdj,"Knockholt, Scotts Lodge (opp)",51.30058,0.08625,NaN,3.0,POINT (0.08625 51.30058)


In [63]:
# Birmingham Census Data

birmingham_map_path = (os.path.join
                   (os.getcwd(),
                    'data',
                    'Birmingham_merged_census_BoundaryData',
                    'england_oac_2011.shp'))

birmingham_census_geo_df = geo_df_from_geospatialfile(birmingham_map_path)


In [70]:
# There is a problem with the "label" column as it contains multiple ONS GSS codes for each observation


def insert_space_middle(str):
    "A function to insert a space into a string after every 9th digit"
    return " ".join([s[i*9:i*9+9] for i in range(len(s)//9)])

birmingham_census_geo_df.label = birmingham_census_geo_df.label.apply(insert_space_middle)

# birmingham_census_geo_df.label.str.split(expand=True)
birmingham_census_geo_df.label.explode()

0       E08000025 E02001922 E01009286 E00047060
1       E08000025 E02001922 E01009286 E00047060
2       E08000025 E02001922 E01009286 E00047060
3       E08000025 E02001922 E01009286 E00047060
4       E08000025 E02001922 E01009286 E00047060
                         ...                   
3218    E08000025 E02001922 E01009286 E00047060
3219    E08000025 E02001922 E01009286 E00047060
3220    E08000025 E02001922 E01009286 E00047060
3221    E08000025 E02001922 E01009286 E00047060
3222    E08000025 E02001922 E01009286 E00047060
Name: label, Length: 3223, dtype: object

In [ ]:
def get_and_save_geo_dataset(url, localpath, filename):
    """Fetches a geodataset from a web resource and saves it to the local data/ directory
    
    Args:
        filename (string): the name of file as it should be saved locally
        url (string): URL of the web resource containing 
        localpath (string):
    Returns:
        json data as dict"""
    file = requests.get(url).json()
    full_path = os.path.join(localpath, filename)
    with open(full_path, 'w') as dset:
        json.dump(file, dset)
    return file

# url = "https://raw.githubusercontent.com/ONSvisual/topojson_boundaries/master/LSOA.json"
birmingham_json_url = 'https://mapit.mysociety.org/area/2514/children' 

birmingahm_geo_dataset = get_and_save_geo_dataset(birmingham_json_url,
                                                  './data/',
                                                  "birmingham_geo_dataset.json")

# path_to_local_file = os.path.join('data', 'birmingham.json')

# with open(path_to_local_file, 'ab') as geo_file:
#             geo_file.write(birmingahm_geo_dataset.content)
    
# # data = requests.get(url)
# # file_path = os.path.join('data', 'LSOA.json')

# # #     gdf = gpd.GeoDataFrame(geo_file.json())

# # # # df = pd.read_json(io.BytesIO(file.content))

# # # gdf = gpd.GeoDataFrame(open(file_path).json())

# # # type(file_path)
# # # file_path
# # df = gpd.read_file(open(file_path, 'rb'), driver='GeoJSON')

birmingahm_geo_dataset


In [8]:

uk_census_df = geo_df_from_geospatialfile(path_to_file, crs="EPSG:4326")

NameError: name 'path_to_file' is not defined

In [9]:
# geo_df_from_geospatialfile(os.path.join
#                            (os.getcwd(),
#                             'data',
#                             'birmingham_geo_dataset.json'))
    
birmingham_df = pd.DataFrame.from_dict(birmingahm_geo_dataset).T
gss_code_cols = pd.DataFrame.from_dict(birmingahm_geo_dataset).T.codes.apply(pd.Series).drop("ons", axis=1) 
birmingham_df = birmingham_df.join(gss_code_cols, on=index).drop(["codes", "all_names"], axis=1) 

NameError: name 'birmingahm_geo_dataset' is not defined

In [10]:
birmingham_df

NameError: name 'birmingham_df' is not defined

In [20]:
# Filter for ward

ward_stops_geo_df = gpd.sjoin(london_stops_geo_df, Brent, how='left', op='within')
ward_stops_geo_df.head()

# Drop all rows where id (from polygon) is NaN, that is, where the point is not in the polygon

ward_stops_geo_df = ward_stops_geo_df[ward_stops_geo_df['index_right'].notna()]
ward_stops_geo_df

# Drop all row names from join (so we can reuse)

ward_stops_geo_df = ward_stops_geo_df[stops_geo_df.columns.to_list()]
ward_stops_geo_df

NameError: name 'Brent' is not defined

3:80: E501 line too long (82 > 79 characters)
6:80: E501 line too long (94 > 79 characters)


In [21]:
## Making a centroid 
centrepoint = ward_polygon.centroid

fig, ax = plt.subplots()
_ = ward_polygon.plot(ax=ax, facecolor='gold')
_ = ward_stops_geo_df.plot(ax=ax, color='red', markersize=2, alpha=0.1)
_ = centrepoint.plot(ax=ax, color='pink', markersize=45) ## added the centroid into the plot
plt.tight_layout()

## great, this works!

NameError: name 'ward_polygon' is not defined

1:1: E266 too many leading '#' for block comment
1:21: W291 trailing whitespace
7:57: E261 at least two spaces before inline comment
7:58: E262 inline comment should start with '# '
7:80: E501 line too long (92 > 79 characters)
10:1: E266 too many leading '#' for block comment
